In [1]:
import sys
sys.path.insert(0, '../asyncroscopy/')
from clients.tem_client import TEMClient
from utils import start_server
import matplotlib.pyplot as plt

import pyTEMlib
from pyTEMlib import probe_tools as pt

You don't have igor2 installed.     If you wish to open igor files, you will need to install it     (pip install igor2) before attempting.
You don't have gwyfile installed.     If you wish to open .gwy files, you will need to      install it (pip install gwyfile) before attempting.
Symmetry functions of spglib enabled
Qt dialogs are not available


In [2]:
# connecting the TEMClient to the central (async) server
tem = TEMClient.connect(host='localhost',port=9000)

Connecting to central server localhost:9000...
Connected to central server.


In [3]:
'''Real server:'''
# tem.send_command_beta(destination = 'Ceos', command = 'getInfo', args = {})
# tem.send_command_beta(destination = 'Ceos', command = 'runTableau', args = {"tabType": 'Fast', "angle": 18})
# tem.send_command_beta(destination = 'Ceos', command = 'correctAberration', args = {"name": 'C10', "target": 0})

'''Twin server:''' # different args
tem.send_command_beta(destination = 'Ceos', command = 'getInfo', args = {})
# tem.send_command_beta(destination = 'Ceos', command = 'runTableau', args = {})
# tem.send_command_beta(destination = 'Ceos', command = 'correctAberration', args = {"name": 'C10', "target": 0})


[client] sending: Ceos_getInfo 
[client] sent: Ceos_getInfo 


'CEOS Digital Twin Server'

In [4]:
aberrations = pt.get_target_aberrations("Spectra300", 60000)
pt.print_aberrations(aberrations)

In [5]:
tem.send_command_beta(destination = 'Ceos', command = 'uploadAberrations',
                      args = aberrations)

[client] sending: Ceos_uploadAberrations C10=0 C12a=0 C12b=0.38448128113770325 C21a=-68.45251255685642 C21b=64.85359774641199 C23a=11.667578600494137 C23b=-29.775627778458194 C30=123 C32a=95.3047364258614 C32b=-189.72105710231244 C34a=-47.45099594807912 C34b=-94.67424667529909 C41a=-905.31842572806 C41b=981.316128853203 C43a=4021.8433526960034 C43b=131.72716642732158 C45a=-4702.390968272048 C45b=-208.25028574642903 C50=552000.0 C52a=-0.0 C52b=0.0 C54a=-0.0 C54b=-0.0 C56a=-36663.643489934424 C56b=21356.079837905396 acceleration_voltage=200000 FOV=34.241659495148205 Cc=1000000.0 convergence_angle=30 wavelength=0.0025079340450548005
[client] sent: Ceos_uploadAberrations C10=0 C12a=0 C12b=0.38448128113770325 C21a=-68.45251255685642 C21b=64.85359774641199 C23a=11.667578600494137 C23b=-29.775627778458194 C30=123 C32a=95.3047364258614 C32b=-189.72105710231244 C34a=-47.45099594807912 C34b=-94.67424667529909 C41a=-905.31842572806 C41b=981.316128853203 C43a=4021.8433526960034 C43b=131.7271664273

'Aberrations Loaded'